In [72]:
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 1. 데이터 
datasets = (load_iris())

x = pd.DataFrame(datasets["data"])
y = pd.DataFrame(datasets["target"])


In [73]:
# print(datasets.colums) 
print(x.shape,y.shape)    
print(y)
# 분류모델이기 때문에 y를 원핫 인코딩변환 해주어야합니다.
y = to_categorical(y)
# print(x.describe)

(150, 4) (150, 1)
     0
0    0
1    0
2    0
3    0
4    0
..  ..
145  2
146  2
147  2
148  2
149  2

[150 rows x 1 columns]


In [74]:

from sklearn.preprocessing import MinMaxScaler,StandardScaler

x_train, x_test, y_train,y_test = train_test_split(x,y,
                                                   train_size=0.8,
                                                   shuffle = True,
                                                #  stratify 는 데이터 불균형을 해결해줌
                                                   stratify=y
                                                #    random_state=21
                                                   )


scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
# x_test 를 스케일링
x_test = scaler.transform(x_test)

print(y_train[:5])
print("shape : ",y_train.shape)

[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
shape :  (120, 3)


In [75]:
model = Sequential([
    Dense(32,activation="relu",input_shape=(4,)),
    Dense(64,activation="relu") ,
    Dense(128,activation="relu") ,
    Dense(64,activation="relu") ,
    Dense(32,activation="relu") ,
    Dense(16,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(3,activation="softmax")
])

In [76]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
min_delta=0.01, patience=5, 
verbose=1, 
mode='min')

# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=100, batch_size=32, 
                 validation_split=0.2,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


Epoch 1/100
3/3 [==============================] - 0s 66ms/step - loss: 1.1159 - accuracy: 0.3021 - val_loss: 1.0166 - val_accuracy: 0.4583
Epoch 2/100
3/3 [==============================] - 0s 14ms/step - loss: 1.0306 - accuracy: 0.3021 - val_loss: 0.9880 - val_accuracy: 0.5833
Epoch 3/100
3/3 [==============================] - 0s 13ms/step - loss: 0.9809 - accuracy: 0.5417 - val_loss: 0.9513 - val_accuracy: 0.4167
Epoch 4/100
3/3 [==============================] - 0s 14ms/step - loss: 0.9333 - accuracy: 0.6458 - val_loss: 0.8933 - val_accuracy: 0.9583
Epoch 5/100
3/3 [==============================] - 0s 13ms/step - loss: 0.8835 - accuracy: 0.8438 - val_loss: 0.8661 - val_accuracy: 0.5417
Epoch 6/100
3/3 [==============================] - 0s 14ms/step - loss: 0.8351 - accuracy: 0.6875 - val_loss: 0.8364 - val_accuracy: 0.5417
Epoch 7/100
3/3 [==============================] - 0s 14ms/step - loss: 0.7889 - accuracy: 0.7292 - val_loss: 0.7660 - val_accuracy: 0.8333
Epoch 8/100
3/3 [===

In [77]:
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


1/1 [==============================] - 0s 105ms/step - loss: 0.5749 - accuracy: 0.8333
loss: 0.5749152302742004 
acc : 0.8333333134651184


In [78]:
from sklearn.metrics import accuracy_score
y_predict = model.predict(x_test)

# predict 의 값은 총 3개가 나오는데 softmax 특성상 sum 의 값은 1이 됨
print(y_predict[:5])

# argmax 는 x 를 최대값으로 만들어주는 입력을 구하는함수
y_predict = np.argmax(y_predict,axis=1)
# y_predict = to_categorical(y_predict,3)
print(y_predict[:15])
# y_test 를 원핫 인코딩 해제 해야함
y_test = np.argmax(y_test,axis=1)
print(y_test[:15])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print(acc)

[[0.13439892 0.28396147 0.5816396 ]
 [0.7201937  0.17402053 0.10578574]
 [0.69567347 0.18856126 0.11576525]
 [0.09959769 0.26624107 0.63416123]
 [0.64080876 0.2161147  0.1430765 ]]
[2 0 0 2 0 2 0 2 0 2 0 1 2 2 1]
[1 0 0 2 0 2 0 1 0 2 0 1 2 1 1]
0.8333333333333334
